Connected to .venv39 (3.9.22) (Python 3.9.22)

In [ ]:
"""
Importing the necessary packages.............
"""

import os
import torch
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import arrow
from datetime import datetime

# from utils.config import config
from torch.utils.data import DataLoader

ROOT = Path(__file__).resolve().parents[2]  # tests/ -> project root
sys.path.insert(0, str(ROOT))
print(ROOT)

from configs.utils import config

# -------------************---------------------------------
# loading all the data loaders here we will load the final data in torch.
from Dataloader.Modis_Data_loader.PM25_data_loader_analysis import Modis_data_loader
from Dataloader.Modis_Data_loader.PM25_data_loader_analysis import PM_25_dataloader
from Dataloader.Modis_Data_loader.PM25_data_loader_analysis import (
    combine_the_data_frames,
)
from Dataloader.Modis_Data_loader.PM25_data_loader_analysis import Training_data_loader
from locationencoder.final_location_encoder import Geospatial_Encoder
from Dataloader.Modis_Data_loader.torch_data_loader import (
    AirQualityDataset_latlon_AOD_PM25,
    AirQualityDataset_latlon_AOD,
    AirQualityDataset_latlon_PM25,
    AirQualityDataset_latlon,
)

# importing config files
from Dataloader.Modis_data_analysis.final_loader import (
    Final_Air_Quality_Dataset_pipeline_latlon_AOD_PM25,
    Final_Air_Quality_Dataset_pipeline_latlon_AOD,
    Final_Air_Quality_Dataset_pipeline_latlon_PM25,
    Final_Air_Quality_Dataset_pipeline_latlon,
)

# importing the loss function
from loss_functions import LossFunctions


# -------------************--------------------------------
# Creating all the instance here

instance_latlon_AOD_PM25 = Final_Air_Quality_Dataset_pipeline_latlon_AOD_PM25(config)
instance_latlon_AOD = Final_Air_Quality_Dataset_pipeline_latlon_AOD(config)
instance_latlon_PM25 = Final_Air_Quality_Dataset_pipeline_latlon_PM25(config)
intance_latlon = Final_Air_Quality_Dataset_pipeline_latlon(config)


# final data with latitude, longitude, AOD and PM2.5
instance_latlon_AOD_PM25.modis_data_sets()
instance_latlon_AOD_PM25.stations_data_sets()
instance_latlon_AOD_PM25.PM_25_data()
instance_latlon_AOD_PM25.training_data()
instance_latlon_AOD_PM25.Torch_data()
final_data_latlong_AOD_PM25 = instance_latlon_AOD_PM25.full_pipeline()
len(final_data_latlong_AOD_PM25[0])

final_data_latlong_AOD_PM25[1]
# final data with latitude, longitude and AOD

instance_latlon_AOD.modis_data_sets()
instance_latlon_AOD.stations_data_sets()
instance_latlon_AOD.PM_25_data()
instance_latlon_AOD.training_data()
instance_latlon_AOD.Torch_data()
final_data_latlong_AOD = instance_latlon_AOD.full_pipeline()
final_data_latlong_AOD[0]
final_data_latlong_AOD[1][0]

# final data with latitude, longitude  and PM2.5
instance_latlon_PM25.modis_data_sets()
instance_latlon_PM25.stations_data_sets()
instance_latlon_PM25.PM_25_data()
instance_latlon_PM25.training_data()
instance_latlon_PM25.Torch_data()
final_data_latlong_PM25 = instance_latlon_PM25.full_pipeline()
final_data_latlong_PM25[0].shape
final_data_latlong_PM25[1]

# final data with latitude and longitude
intance_latlon.modis_data_sets()
intance_latlon.stations_data_sets()
intance_latlon.PM_25_data()
intance_latlon.training_data()
intance_latlon.Torch_data()
final_data_latlong = intance_latlon.full_pipeline()
final_data_latlong[0].shape
final_data_latlong[1][0]


# -------------************--------------------------------
"""
Final training data.................
"""
final_data_latlong_AOD_PM25

final_data_latlong_AOD_PM25
final_data_latlong_AOD[0].shape
final_data_latlong_PM25[0].shape
final_data_latlong[0].shape

for i, t in enumerate(final_data_latlong_AOD_PM25):
    print(i, t.shape)
# final data sets wit x, y in torch tensor form
final_data_latlong_AOD_PM25[0].shape
x, y = final_data_latlong_AOD
x.shape[0]
final_data_latlong_PM25
final_data_latlong


# -------------************--------------------------------

# training the model in different sdata sets and storing the weights.

# -------------************--------------------------------

# importing the model from model file.
"""
Importing the model................
"""

from src.Models.neural_process import NeuralProcess

# self, x_c_dim, y_c_dim, x_t_dim, y_t_dim, hidden_dim, latent_dim
# self, x_target_dim, z_dim, hidden_dim, y_target_dim
# self, x_c_dim, y_c_dim, x_t_dim, y_t_dim, hidden_dim, latent_dim

# Creating the models those are compatible with the all kinds of inputs.

# -------------************---------------------------------

"""
Final models for training........... making attributes from the class
"""

model_latlon_AOD_PM25 = NeuralProcess(128, 2, 128, 2, 128, 128)
model_latlon_AOD = NeuralProcess(127, 2, 127, 2, 128, 128)
model_latlon_PM25 = NeuralProcess(127, 2, 127, 2, 128, 128)
model_latlon = NeuralProcess(126, 2, 127, 2, 128, 128)


"""
Here we will import the loss function...........
"""
# self, beta, learning_rate, stepsize, Number_of_steps, device#
Loss = LossFunctions()


"""
Now we have model, loss function, data from here we can build the model trainer that will train the model with the tensor board.

The first & second class --: will devide the data in to chunks and also train, val and test this will create a final data loader. 

Third class --: this class will create the loop for train the network. 

Fourth class --: will validate the data set 

Fifth step --: will evaluate the model.

"""

# ------------ The first step is to devide the data in terms of context and test sets.


from optimizer_utils import context_target_split

C_T_data = context_target_split(
    final_data_latlong_AOD_PM25[0].unsqueeze(0),
    final_data_latlong_AOD_PM25[1].unsqueeze(0),
)

C_T_data[0].shape
C_T_data[1].shape
C_T_data[2].shape
C_T_data[3].shape

final_data_latlong_AOD_PM25[0].unsqueeze(0)[0]

""" self,
        model,
        optimizer,
        loss_fn,
        device,
        log_dir="./logs",
        checkpoint_dir="./checkpoints",
"""

# Importing the optimizer
import torch.optim as optim

Optimizer = optim.Adam(model_latlon_AOD_PM25.parameters(), lr=1e-4)

# Importing the trainer

final_data_latlong_AOD_PM25[0].__getitem__(10)
# First data ser priority will be
# def train_epoch(self, dataloader, epoch_idx):

from optimizer_utils import neural_process_data
# now to start training we need a data-loader that

NeuralProcessData_latlon_AOD_PM25 = neural_process_data(
    final_data_latlong_AOD_PM25[0],
    final_data_latlong_AOD_PM25[1],
    num_points_per_task=200,
)

X_task, Y_task = NeuralProcessData_latlon_AOD_PM25[0]
X_task.shape
Y_task.shape

/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
{'dataset_num_1': 1, 'dataset_num_2': 2, 'dataset_num_3': 3, 'exp_name': 'aod_pm25_prediction', 'input_type_1': 1, 'input_type_2': 2, 'input_type_3': 3, 'input_type_4': 4, 'target_type': ['AOD', 'PM2.5'], 'input_vars': {1: ['latitude', 'longitude', 'AOD', 'PM2.5'], 2: ['latitude', 'longitude', 'AOD'], 3: ['latitude', 'longitude', 'PM2.5'], 4: ['latitude', 'longitude']}}


/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis/.venv39/lib/python3.9/site-packages/lightning/fabric/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
Extracting daily Pm2.5 data..
 daily_PM_data shape: (40, 1095)
Fusing AOD data from multiple years...
 AOD_final shape: (1350, 1098)
Finding nearest neighbours for PM2.5 stations...
nearest_neighbours shape: (40, 1098)
Cleaning data and handling missing values...
Cleaned nearest_neighbours shape: (40, 1097)
Preparing final training data...
final_training_data shape: (15667, 5)
Extracting daily Pm2.5 data..
 daily_PM_data shape: (40, 1095)
Fusing AOD data from multiple years...
 AOD_final shape: (1350, 1098)
Finding nearest neighbours for PM2.5 stations...
nearest_neighbours shape: (40, 1098)
Cleaning data and handling missing values...
Cleaned nearest_neighbours shape: (40, 1097)
Preparing final training data...
final_training_data shape: (15667, 5)
Extracting daily Pm2.5 d

torch.Size([200, 2])

In [ ]:
final_data_latlong_AOD_PM25

(tensor([[ 0.8213, -1.2706, -0.4533,  ..., -0.2197,  0.9654,  1.2592],
         [ 0.8213, -1.2706, -0.4533,  ..., -0.2197, -0.7220, -0.0482],
         [ 0.8213, -1.2706, -0.4533,  ..., -0.2197, -0.3033, -0.1231],
         ...,
         [ 0.8200, -1.2679, -0.4557,  ..., -0.2200, -0.5858, -0.2545],
         [ 0.8200, -1.2679, -0.4557,  ..., -0.2200,  0.4649, -0.3712],
         [ 0.8200, -1.2679, -0.4557,  ..., -0.2200, -0.4296,  0.0278]],
        grad_fn=<CatBackward0>),
 tensor([[ 0.9654,  1.2592],
         [-0.7220, -0.0482],
         [-0.3033, -0.1231],
         ...,
         [-0.5858, -0.2545],
         [ 0.4649, -0.3712],
         [-0.4296,  0.0278]]))

In [ ]:
final_data_latlong_AOD_PM25[0]

tensor([[ 0.8213, -1.2706, -0.4533,  ..., -0.2197,  0.9654,  1.2592],
        [ 0.8213, -1.2706, -0.4533,  ..., -0.2197, -0.7220, -0.0482],
        [ 0.8213, -1.2706, -0.4533,  ..., -0.2197, -0.3033, -0.1231],
        ...,
        [ 0.8200, -1.2679, -0.4557,  ..., -0.2200, -0.5858, -0.2545],
        [ 0.8200, -1.2679, -0.4557,  ..., -0.2200,  0.4649, -0.3712],
        [ 0.8200, -1.2679, -0.4557,  ..., -0.2200, -0.4296,  0.0278]],
       grad_fn=<CatBackward0>)

In [ ]:
final_data_latlong_AOD_PM25

(tensor([[ 0.8213, -1.2706, -0.4533,  ..., -0.2197,  0.9654,  1.2592],
         [ 0.8213, -1.2706, -0.4533,  ..., -0.2197, -0.7220, -0.0482],
         [ 0.8213, -1.2706, -0.4533,  ..., -0.2197, -0.3033, -0.1231],
         ...,
         [ 0.8200, -1.2679, -0.4557,  ..., -0.2200, -0.5858, -0.2545],
         [ 0.8200, -1.2679, -0.4557,  ..., -0.2200,  0.4649, -0.3712],
         [ 0.8200, -1.2679, -0.4557,  ..., -0.2200, -0.4296,  0.0278]],
        grad_fn=<CatBackward0>),
 tensor([[ 0.9654,  1.2592],
         [-0.7220, -0.0482],
         [-0.3033, -0.1231],
         ...,
         [-0.5858, -0.2545],
         [ 0.4649, -0.3712],
         [-0.4296,  0.0278]]))

In [ ]:
final_data_latlong

(tensor([[ 1.2384,  0.5588, -0.0965,  ...,  0.3799,  0.1579,  0.1248],
         [ 1.2384,  0.5588, -0.0965,  ...,  0.3799,  0.1579,  0.1248],
         [ 1.2384,  0.5588, -0.0965,  ...,  0.3799,  0.1579,  0.1248],
         ...,
         [ 1.2386,  0.5598, -0.0965,  ...,  0.3793,  0.1597,  0.1252],
         [ 1.2386,  0.5598, -0.0965,  ...,  0.3793,  0.1597,  0.1252],
         [ 1.2386,  0.5598, -0.0965,  ...,  0.3793,  0.1597,  0.1252]],
        grad_fn=<CatBackward0>),
 tensor([[  1.2280, 225.3438],
         [  0.5470, 112.8750],
         [  0.7160, 106.4375],
         ...,
         [  0.6020,  95.1299],
         [  1.0260,  85.0938],
         [  0.6650, 119.4167]]))

In [ ]:
x, y = final_data_latlong_AOD

In [ ]:
x.shape[0]

12307

In [ ]:
x.

SyntaxError: invalid syntax (<ipython-input-8-a2885923daf8>, line 1)

In [ ]:
x

tensor([[-0.7793,  1.3130, -0.5150,  ..., -1.1960,  0.8817,  1.2280],
        [-0.7793,  1.3130, -0.5150,  ..., -1.1960,  0.8817,  0.5470],
        [-0.7793,  1.3130, -0.5150,  ..., -1.1960,  0.8817,  0.7160],
        ...,
        [-0.7818,  1.3116, -0.5153,  ..., -1.1917,  0.8828,  0.6020],
        [-0.7818,  1.3116, -0.5153,  ..., -1.1917,  0.8828,  1.0260],
        [-0.7818,  1.3116, -0.5153,  ..., -1.1917,  0.8828,  0.6650]],
       grad_fn=<CatBackward0>)

In [ ]:
from src.Models.neural_process import NeuralProcess

In [ ]:
model_latlon_AOD_PM25 = NeuralProcess(128, 2, 128, 2, 128, 128)
model_latlon_AOD = NeuralProcess(127, 2, 127, 2, 128, 128)
model_latlon_PM25 = NeuralProcess(127, 2, 127, 2, 128, 128)
model_latlon = NeuralProcess(126, 2, 127, 2, 128, 128)

In [ ]:
Loss = LossFunctions()

In [ ]:
from optimizer_utils import context_target_split

In [ ]:
C_T_data = context_target_split(
    final_data_latlong_AOD_PM25[0].unsqueeze(0),
    final_data_latlong_AOD_PM25[1].unsqueeze(0),
)

In [ ]:
C_T_data[0].shape
C_T_data[1].shape
C_T_data[2].shape
C_T_data[3].shape

torch.Size([1, 1000, 2])

In [ ]:
final_data_latlong_AOD_PM25[0].unsqueeze(0)[0]

tensor([[ 0.8213, -1.2706, -0.4533,  ..., -0.2197,  0.9654,  1.2592],
        [ 0.8213, -1.2706, -0.4533,  ..., -0.2197, -0.7220, -0.0482],
        [ 0.8213, -1.2706, -0.4533,  ..., -0.2197, -0.3033, -0.1231],
        ...,
        [ 0.8200, -1.2679, -0.4557,  ..., -0.2200, -0.5858, -0.2545],
        [ 0.8200, -1.2679, -0.4557,  ..., -0.2200,  0.4649, -0.3712],
        [ 0.8200, -1.2679, -0.4557,  ..., -0.2200, -0.4296,  0.0278]],
       grad_fn=<SelectBackward0>)

In [ ]:
import torch.optim as optim

In [ ]:
Optimizer = optim.Adam(model_latlon_AOD_PM25.parameters(), lr=1e-4)

In [ ]:
final_data_latlong_AOD_PM25[0].__getitem__(10)

tensor([ 8.2131e-01, -1.2706e+00, -4.5331e-01,  2.9036e-01, -1.2760e+00,
        -3.9974e-01,  1.8594e+00,  1.1703e+00, -1.8606e-05,  5.2758e-01,
         1.3764e+00, -1.1656e+00,  8.9693e-01,  1.8188e-01,  2.2443e+00,
        -1.0425e+00,  3.8400e-01,  1.2789e+00,  3.3677e-01, -1.8441e-01,
         2.4959e-01,  1.2210e+00, -2.5053e-01, -4.3350e-01,  1.8787e-01,
         5.5356e-01,  2.6125e-01,  5.6281e-01, -3.3981e-01,  1.6208e+00,
        -2.6066e-01,  5.3954e-01,  5.3861e-01, -7.1402e-01,  1.0262e-01,
        -4.3640e-01,  4.7413e-01, -8.6633e-01,  1.5694e-01,  3.8956e-02,
        -3.2202e-02, -4.6373e-01, -2.5426e-01, -9.5811e-01, -2.9337e-03,
        -3.0596e-01,  5.8505e-01,  3.5150e-01, -9.1359e-02,  5.6125e-01,
         4.2494e-01, -8.4200e-01,  6.8421e-01, -2.4915e-01, -3.1594e-02,
         7.5370e-01, -5.4094e-01, -1.1469e+00,  6.3201e-01, -4.4623e-01,
        -1.9939e+00,  9.2838e-01, -1.9337e+00,  5.4995e-01,  9.0804e-03,
         5.9840e-01,  7.6809e-01,  4.4503e-01, -1.7

In [ ]:
from optimizer_utils import neural_process_data

In [ ]:
NeuralProcessData_latlon_AOD_PM25 = neural_process_data(
    final_data_latlong_AOD_PM25[0],
    final_data_latlong_AOD_PM25[1],
    num_points_per_task=200,
)

In [ ]:
X_task, Y_task = NeuralProcessData_latlon_AOD_PM25[0]
X_task.shape
Y_task.shape

torch.Size([200, 2])

In [ ]:
dataloader = DataLoader(
    dataset=NeuralProcessData_latlon_AOD_PM25,
    batch_size=16,
    shuffle=True,
    num_workers=0,
)

In [ ]:
Xb, Yb = next(iter(dataloader))
Xb.shape, Yb.shape
from optimizer_utils import NPTrainer

In [ ]:
Xb.shape, Yb.shape

(torch.Size([16, 200, 128]), torch.Size([16, 200, 2]))

In [ ]:
Xb.shape, Yb.shape

(torch.Size([16, 200, 128]), torch.Size([16, 200, 2]))

In [ ]:
Training = NPTrainer(
    model=model_latlon_AOD_PM25,
    optimizer=Optimizer,
    loss_fn=Loss,
    device="cpu",
    log_dir="./logs",
    checkpoint_dir="./checkpoints",
)

In [ ]:
for epoch in range(10):
    Training.train_epoch(dataloader, epoch)

Epoch 9: 100%|██████████| 770/770 [00:11<00:00, 65.68it/s, Loss=nan]


In [ ]:
dataloader.dataset

In [ ]:
dataloader.dataset

In [ ]:
dataloader.dataset

In [ ]:
Xb.shape, Yb.shape

(torch.Size([16, 200, 128]), torch.Size([16, 200, 2]))